In [43]:
# Code based on Udemy lecture video on NLP

In [1]:
import pandas as pd
npr = pd.read_csv('npr.csv') # Example dataset used in the lecture

In [2]:
# Describe the df
npr.describe()

,Article
count,11992
unique,11991
top,"Washington state has released an estimated 3, ..."
freq,2


In [3]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [5]:
npr['Article'][0] # full text of articles

'In the Washington of 2016, even when the policy can be bipartisan, the politics cannot. And in that sense, this year shows little sign of ending on Dec. 31. When President Obama moved to sanction Russia over its alleged interference in the U. S. election just concluded, some Republicans who had long called for similar or more severe measures could scarcely bring themselves to approve. House Speaker Paul Ryan called the Obama measures ”appropriate” but also ”overdue” and ”a prime example of this administration’s ineffective foreign policy that has left America weaker in the eyes of the world.” Other GOP leaders sounded much the same theme. ”[We have] been urging President Obama for years to take strong action to deter Russia’s worldwide aggression, including its   operations,” wrote Rep. Devin Nunes,  . chairman of the House Intelligence Committee. ”Now with just a few weeks left in office, the president has suddenly decided that some stronger measures are indeed warranted.” Appearing 

In [6]:
npr.shape

(11992, 1)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
# max_df --- ignores words that appear in >90% of documents
# min_d --- has to show up X% of time or X # of times
cv = CountVectorizer(max_df=0.9, min_df=2, stop_words='english')

In [20]:
cv.get_feature_names()[8000] # showing the result of countvectorizer

'canopy'

In [9]:
dtm = cv.fit_transform(npr['Article'])

In [10]:
dtm # this is document x feature names (you can return this from cv.get_feature_names())

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.decomposition import LatentDirichletAllocation
# n_components --- important to set
LDA = LatentDirichletAllocation(n_components=7,random_state=42)

In [12]:
LDA.fit(dtm) # this may take a while to train

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=7, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [21]:
# Steps for the analysis using LDA 
# Grab the vocabulary of words
# Grab the topics (which use those words)
# Grab the highest probability words per topic

len(LDA.components_)

7

In [23]:
type(LDA.components_)

numpy.ndarray

In [26]:
# first component probability for each of the words
LDA.components_[0]

array([8.64332806e+00, 2.38014333e+03, 1.42900522e-01, ...,
       1.43006821e-01, 1.42902042e-01, 1.42861626e-01])

In [27]:
single_topic = LDA.components_[0]
single_topic.argsort()

array([ 2475, 18302, 35285, ..., 22673, 42561, 42993])

In [30]:
# note: how argsort works is that given an array, it returns the index
# of the smallest item and then second smallest, etc
import numpy as np
arr = np.array([5,3,1])
arr.argsort() # expect array([2,1,0])

array([2, 1, 0])

In [32]:
# Top 10 values aka last 10 values of argsort()
single_topic.argsort()[-10:] 

array([33390, 36310, 21228, 10425, 31464,  8149, 36283, 22673, 42561,
       42993])

In [33]:
top_ten_words = single_topic.argsort()[-10:] 
for index in top_ten_words:
    print(cv.get_feature_names()[index])

new
percent
government
company
million
care
people
health
said
says


In [34]:
top_thirty_words = single_topic.argsort()[-30:] 
for index in top_thirty_words:
    print(cv.get_feature_names()[index])

jobs
workers
administration
years
according
law
plan
states
business
pay
president
state
tax
insurance
trump
companies
money
year
federal
000
new
percent
government
company
million
care
people
health
said
says


In [37]:
# Now, we can do this for each topoic
for i, topic in enumerate(LDA.components_):
    print(f"The TOP 30 words for each topic #{i}")
    print([cv.get_feature_names()[index] for index in topic.argsort()[-30:]])
    print('\n')
    print('\n')
    print('\n')

The TOP 30 words for each topic #0
['jobs', 'workers', 'administration', 'years', 'according', 'law', 'plan', 'states', 'business', 'pay', 'president', 'state', 'tax', 'insurance', 'trump', 'companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']






The TOP 30 words for each topic #1
['statement', 'did', 'investigation', 'state', 'department', 'russian', 'officials', 'press', 'new', 'country', 'white', 'according', 'attack', 'reported', 'war', 'military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']






The TOP 30 words for each topic #2
['ve', 'place', 'called', 'come', 'used', 'long', 'small', 'local', 'said', 'life', 'little', 'know', 'don', 'year', 'make', 'way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']






The TOP 30 word

In [38]:
# Now, needs to assign topics
topic_results = LDA.transform(dtm)

In [39]:
topic_results[0]

array([1.61040465e-02, 6.83341493e-01, 2.25376318e-04, 2.25369288e-04,
       2.99652737e-01, 2.25479379e-04, 2.25497980e-04])

In [40]:
topic_results.shape

(11992, 7)

In [41]:
topic_results[0].argmax() # returns the index of the largest probability topic

1

In [42]:
# Note that, I can use argsort but use -1 again
topic_results[0].argsort()[-1]

1

In [44]:
# Now, assign topics
npr['Topic'] = topic_results.argmax(axis=1)
npr.head(20)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1
4,"From photography, illustration and video, to d...",2
5,I did not want to join yoga class. I hated tho...,3
6,With a who has publicly supported the debunk...,3
7,"I was standing by the airport exit, debating w...",2
8,"If movies were trying to be more realistic, pe...",3
9,"Eighteen years ago, on New Year’s Eve, David F...",2
